# Fisher Vector Encoding Basics

GMM with $K$ components and diagonal covariance matrix:
$$
\begin{equation}
    p(\vec{x}_n|\Theta) = \sum_{k=1}^{K} \alpha_k p_k(\vec{x}_n|\theta_k) \\
	p_k(\vec{x}_n|\vec{\mu}_k, \vec{\sigma}_k)
	= \dfrac{1}{\prod_{d=1}^{D}\sqrt{2\pi} \sigma_{k,d}}
    \exp\left(-\sum_{d=1}^{D}\frac{(x_{n,d} - \mu_{k,d})^2}{2 (\sigma_{k,d})^2}\right)\quad.
\end{equation}
$$

FVE based on this GMM:

$$
\begin{align}
	\mathcal{F}_{\mu_{k,d}}(\mathcal{X}_I)
		&= \dfrac{1}{\sqrt{N_I\alpha_k}}\sum_{n=1}^{N_I} w_{n,k} \left(\dfrac{x_{n,d} - \mu_{k,d}}{\sigma_{k,d}}\right)\quad,
		\\
	\mathcal{F}_{\sigma_{k,d}}(\mathcal{X}_I)
		&= \dfrac{1}{\sqrt{2N_I\alpha_k}}\sum_{n=1}^{N_I} w_{n,k} \left(\dfrac{(x_{n,d} - \mu_{k,d})^2}{(\sigma_{k,d})^2} - 1\right)\quad,
\end{align}
$$ 

with soft assignment

$$
\begin{equation}
	w_{n,k} = \dfrac{\alpha_k p_k(\vec{x}_n|\theta_k)}{\sum_{\ell=1}^{K}\alpha_\ell p_\ell(\vec{x}_n|\theta_\ell)}\quad.
	\label{eq:soft_assignment}
\end{equation}
$$

The gradients of this encoding w.r.t. an input feature $x_{n,d_*}$ are:

$$
\begin{align}
	\frac{\partial\mathcal{F}_{\mu_{k,d}}(\mathcal{X}_I)}{\partial{x_{n,d_*}}}
		&= \dfrac{1}{\sqrt{N_I\alpha_k}}\left[\frac{\partial{w_{n,k}}}{\partial{x_{n,d_*}}}\left(\dfrac{x_{n,d} - \mu_{k,d}}{\sigma_{k,d}}\right) + \delta_{d,d_*}\frac{w_{n,k}}{\sigma_{k,d_*}}\right] \quad,\\
	\frac{\partial\mathcal{F}_{\sigma_{k,d}}(\mathcal{X}_I)}{\partial{x_{n,d_*}}}
		&= \dfrac{1}{\sqrt{2N_I\alpha_k}}\left[
		     \frac{\partial{w_{n,k}}}{\partial{x_{n,d_*}}}\left(\dfrac{(x_{n,d} - \mu_{k,d})^2}{(\sigma_{k,d})^2} - 1\right)\right. \nonumber \\
		     & \phantom{==\dfrac{1}{\sqrt{2N_I\alpha_k}}}\left.+\delta_{d,d_*}\frac{2 w_{n,k}(x_{n,d_*}-\mu_{k,d_*})}{(\sigma_{k,d_*})^2}\right]\quad.
\end{align}
$$
In both equations, we use $\delta_{d,d_*}$ to denote the Kronecker delta being $1$ if $d=d_*$ and $0$ else, as well as the derivative of $w_{n,k}$ w.r.t. $x_{n,d_*}$ that is given by:
$$
\begin{equation}
    \frac{\partial{w_{n,k}}}{\partial{x_{n,d_*}}} = w_{n,k}\left(-\frac{(x_{n,d_*}-\mu_{k,d_*})}{(\sigma_{k,d_*})^2} + \sum_{\ell=1}^K w_{n,\ell}\frac{(x_{n,d_*}-\mu_{\ell,d_*})}{(\sigma_{\ell,d_*})^2} \right)\quad.
\end{equation}
$$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import chainer
import cv2
import abc
import sys

from chainer import functions as F

from ipywidgets import interactive
from tabulate import tabulate

from matplotlib import pyplot as plt
from dataclasses import dataclass
from typing import Tuple

from fve_layer.backends.chainer import links

print(f"Chainer version: {chainer.__version__}")

chainer.config.train = False

Chainer version: 7.7.0


# Arguments

In [3]:
@dataclass
class Args:
    n_components:       int     = 3
    n_dims:             int     = 2
        
    init_mu:            float   = 5 # ~ U(-init_mu, init_mu)
    rnd_mu:             bool    = True
    init_sig:           float   = 1
    eps:                float   = 1e-2
        
    x_range:            Tuple[float, float] = (-10, 10)
    y_range:            Tuple[float, float] = (-10, 10)
    n_samples:          int     = 25
        
    
    seed:               int     = 42


# Data and Layer initializers

In [4]:
def new_layer(args: Args, mu=None, sig=None, N=None):
    
    init_mu = args.init_mu if mu is None else mu
    n_comp = args.n_components if N is None else N
    
    # the random mu initialization (uniform in [-init_mu, init_mu]) is realized by the layer
    if not args.rnd_mu:
        init_mu = np.full((args.n_dims, n_comp), init_mu, dtype=np.float32)
        

    layer = links.FVELayer_noEM(
        in_size=args.n_dims,
        n_components=n_comp,

        init_mu=init_mu,
        init_sig=args.init_sig if sig is None else sig,

        eps=args.eps
    )
    layer.cleargrads()

    return layer

In [5]:
def new_data_grid(args: Args):
    
    xs,ys = [np.linspace(*r, num=args.n_samples) for r in [args.x_range, args.y_range]]
    grid = np.stack(np.meshgrid(ys, xs), axis=2).astype(np.float32)
    return grid

In [6]:
def generate_data(n_classes, n_samples, *, rnd=None, overlap=False, map_size=1, dtype=np.float32):
    Xs, ys = [], []
    
    if rnd is None:
        rnd = np.random.RandomState()

    _comp_pos = np.linspace(0, 2*np.pi, n_classes, endpoint=False)
    _means = np.vstack([np.cos(_comp_pos), np.sin(_comp_pos)]).T
    _std_factor = 2 if overlap else 1
    _std = np.array([1., 1.]) / n_classes * _std_factor

    for i, mean in enumerate(_means):
        X = rnd.normal(loc=mean, scale=_std, size=(n_samples, 2))
        X = X.astype(dtype)

        X = np.expand_dims(np.expand_dims(X, 1), 1)
        _pad_size = (map_size - 1) // 2
        pad = (_pad_size, _pad_size)
        X = np.pad(X, pad_width=[(0,0), pad, pad, (0,0)], mode="constant")

        noise = rnd.normal(loc=(0, 0), scale=(0.01, 0.01), size=X.shape)
        noise_mask = X==0
        X[noise_mask] = noise[noise_mask]

        X = X.reshape(n_samples, -1, 2)

        y = np.full(shape=n_samples, fill_value=i, dtype=dtype)
        y = np.expand_dims(y, 1)
        y = np.broadcast_to(y, (n_samples, X.shape[1]))

        Xs.append(X)
        ys.append(y)

    Xs = np.vstack(Xs)
    ys = np.vstack(ys)

    return Xs*10, ys, _means*10, _std*10

In [7]:
X, y, *_ = generate_data(4, 20)

In [8]:
X.shape

(80, 1, 2)

# Plotting helper

In [9]:
def _norm(arr, axis=(0,1)):
    _min = arr.min(axis=axis)
    arr[:] -= _min
    _max = arr.max(axis=axis, keepdims=True)
    _max_mask = _max != 0
    if _max_mask.any():
        arr_mask = np.broadcast_to(_max_mask, arr.shape)
        arr[arr_mask] /= np.broadcast_to(_max, arr.shape)[arr_mask]
    
    return _min, np.squeeze(_max)

def _denorm(arr, _min, _max):
    _max_mask = _max != 0
    if _max_mask.any():
        arr = arr.copy()
        arr[_max_mask] *= _max[_max_mask] 

    return arr + _min

def _imshow(ax, im, x_range, y_range, *, title=None, YUV2RGB=False, display_values=5, ndecimals=4):
    assert im.shape[-1] >= 2
    im = getattr(im, "array", im)
    
    _im = np.zeros(im.shape[:-1] + (3,), dtype=im.dtype)
    _im[..., 1:] = im[..., :2].copy()
    
    _min, _max = _norm(_im)
    
    if YUV2RGB:
        _im = cv2.cvtColor(_im, cv2.COLOR_YUV2RGB)

    _norm(_im)
    _im[..., 0] = 0

    ax.imshow(_im.round(ndecimals), extent=x_range + y_range, origin="lower")
    
    if title is not None:
        ax.set_title(title)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    
    
    if display_values < 1:
        return
    h, w, _ = _im.shape
    xs = np.linspace(*x_range, display_values)
    ys = np.linspace(*y_range, display_values)
    _js = np.linspace(0, w-1, display_values).astype(int)
    _is = np.linspace(0, h-1, display_values).astype(int)

    fmt = "\n".join(["{x} | {y}", "R={0: .2f}", "G={1: .2f}", "B={2: .2f}"])
    for y, i in zip(ys, _is):
        for x, j in zip(xs, _js):

            text = fmt.format(*_denorm(_im[i, j], _min, _max), x=x, y=y)
            if i == _is.min():
                verticalalignment = "bottom"
            elif i == _is.max():
                verticalalignment = "top"
            else:
                verticalalignment = "center"

            if j == _js.min():
                horizontalalignment = "left"
            elif j == _js.max():
                horizontalalignment = "right"
            else:
                horizontalalignment = "center"

            ax.text(x,y, s=text, 
                    bbox=dict(facecolor="white", alpha=0.5),
                    horizontalalignment=horizontalalignment,
                    verticalalignment=verticalalignment,
                   )
            ax.scatter(x, y, marker="x", color="white")
                


# Printing helper


## Gradient printing

We compute the actual $\vec{\sigma^2}$ and $\vec{w}$ indirectly from learned network parameters $\vec{s}$ and $\vec{v}$:

$$
\begin{align}
    \sigma^2_i 
        &= \epsilon + \exp(s_i), \quad s \in \mathbb{R} \\
    w_i 
        &= \dfrac{\mathrm{sigmoid}(v_i)}{\sum_{j} \mathrm{sigmoid}(v_j)}, \quad v_i \in \mathbb{R} \\        
\end{align}
$$

In the network, we can only observe the gradients w.r.t. the learned parameters:

$$
\begin{align}
\dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{s}}
    &= \dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{\sigma^2}} \cdot 
    \dfrac{\partial \vec{\sigma^2}}{\partial \vec{s}}\\
\dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{v}}
    &= \dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{w}} \cdot 
    \dfrac{\partial \vec{w}}{\partial \vec{v}}\\
\end{align}
$$

Hence, the actual gradients of the encoding w.r.t. the GMM paramters are 
(some indexing is missing, but it should be straighforward to complement those):


$$
\begin{align}
\dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{\sigma^2}}
    &= \dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{s}} \cdot
        \left(\dfrac{\partial \vec{\sigma^2}}{\partial \vec{s}}\right)^{-1} \\
    &= \dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{s}} \cdot
         \dfrac{1}{\exp(\vec{s})} \\
\dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{w}}
    &= \dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{v}}
        \left(\dfrac{\partial \vec{w}}{\partial \vec{v}}\right)^{-1} \\
    &= \dfrac{\partial \mathcal{F}_\Theta(X)}{\partial \vec{v}} \circ
    \left(\vec{w}\circ\vec{1}^T \circ \left(\left(\mathbf{1} - \vec{1}\circ\vec{w}^T\right) \cdot \left(\vec{1}-\mathrm{sigmoid}(\vec{v}^T)\right)\right)\right)^{-1}
\end{align}
$$

Following code cofirms the computations

In [10]:
shape = (2,3)
data = np.random.randn(*shape)
_sig = chainer.Variable(data, requires_grad=True)
eps = 1e-2
sig = eps + F.exp(_sig)

sig.grad = np.full(sig.shape, fill_value=np.e, dtype=sig.dtype)
_sig.grad = None

with chainer.force_backprop_mode():
    sig.backward(retain_grad=True)

print("==" * 5, "_sig Grad:", "==" * 5)
print("Direct:\n", _sig.grad)
print("Indirect:\n",  (sig.array - eps) * sig.grad)

print("==" * 5, "sig Grad:", "==" * 5)
print("Direct:\n", sig.grad)
print("Indirect:\n", _sig.grad / (sig.array - eps))

========== _sig Grad: ==========
Direct:
 [[ 5.46159784  1.49607285  2.43134807]
 [ 1.55475592  5.87558033 24.14606337]]
Indirect:
 [[ 5.46159784  1.49607285  2.43134807]
 [ 1.55475592  5.87558033 24.14606337]]
========== sig Grad: ==========
Direct:
 [[2.71828183 2.71828183 2.71828183]
 [2.71828183 2.71828183 2.71828183]]
Indirect:
 [[2.71828183 2.71828183 2.71828183]
 [2.71828183 2.71828183 2.71828183]]


In [12]:
shape = (3,)
rnd = np.random.RandomState(42)

data = rnd.randn(*shape).astype(np.float32)
# data = np.ones(shape)
# data = np.arange(shape[0], dtype=float)
_w = chainer.Variable(data, requires_grad=True)

def f(x):
    return F.sigmoid(x) / F.sum(F.sigmoid(x))

def f2(x):
    return F.exp(x) / F.sum(F.exp(x))

def grad(x, y):
    """ Source: http://saitcelebi.com/tut/output/part2.html#back_propagation_phase"""

    y2 = (1 - F.sigmoid(y).array).reshape(-1, 1)
    return grad2(x, y) * y2

def grad2(x, *_):
    """ Source: http://saitcelebi.com/tut/output/part2.html#back_propagation_phase"""
    
    assert x.ndim == 1
    n, = x.shape
    
    y0 = np.matmul(x.reshape(-1, 1), np.ones((1, n)))
    y1 = np.identity(n) - np.matmul(np.ones((n, 1)), x.reshape(1, -1))
    return y0 * y1

def encode(w):
    global rnd
    #return F.sum(w)
    _data0 = rnd.randn(*w.shape).astype(w.dtype)
    #print("_data0:\n", _data0)
    # simulates the soft-assignment
    y = w * _data0
    y /= F.sum(y)
    
    # simulates the encoding
    _data1 = rnd.rand(*w.shape).astype(w.dtype)
    #print("_data1:\n", _data1)
    y = y.array * _data1 / F.sqrt(w)
    return F.sum(y)


if 0:
    w = f(_w)
    grad_matrix = grad(w.array, _w.array)
else:
    w = f2(_w)
    grad_matrix = grad2(w.array, _w.array)

inv_grad_matrix = np.linalg.pinv(grad_matrix)
if not np.allclose(inv_grad_matrix @ grad_matrix, np.eye(shape[0])):
    print(grad_matrix)
    print(inv_grad_matrix)
    print((inv_grad_matrix @ grad_matrix).round(6))
    
_w.grad = None

with chainer.force_backprop_mode():
    encode(w).backward(retain_grad=True)


# the grads must sum up to 0!
print("==" * 5, "_w Grad:", "==" * 5)
print("Direct:\n", _w.grad)
print("Grad sum:\n", _w.grad.sum(), np.isclose(_w.grad.sum(), 0))

ind_grad = grad_matrix @ w.grad
print("Indirect:\n", ind_grad)
print("is close? \n", np.isclose(_w.grad, ind_grad))

print("==" * 5, "w Grad:", "==" * 5)
print("Direct:\n", w.grad)
# ind_grad = np.linalg.solve(grad_matrix, _w.grad)
ind_grad = inv_grad_matrix @ _w.grad

print("Indirect2:\n", ind_grad)
print("is close?:\n", np.isclose(ind_grad, w.grad))

========== _w Grad: ==========
Direct:
 [-0.12147768  0.07729177  0.04418591]
Grad sum:
 0.0 True
Indirect:
 [-0.12147767  0.07729178  0.0441859 ]
is close? 
 [ True  True  True]
========== w Grad: ==========
Direct:
 [-0.17349128  0.54639417  0.25595254]
Indirect2:
 [-0.32712893  0.39275649  0.1023149 ]
is close?:
 [False False False]


In [13]:
def _print(params, data_name="array", headers=["\u03BC", "\u03C3", "w"]):
    
    rows = []
    _get = lambda param: getattr(param, data_name, param)
    p_arrays = [_get(p) for p in params]
    for i, params in enumerate(zip(*p_arrays), 1):
        rows.append([f"Comp #{i}"] + [p.round(6) for p in params])
    print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))

def print_arrays(layer):
    _print([layer.mu.T, layer.sig.T, layer.w], "array")
    
    

In [33]:
def sig_grad(layer):
    return layer._sig.grad / (layer.sig.array - layer.eps)

def w_grad(layer):
    
    """ Source: http://saitcelebi.com/tut/output/part2.html#back_propagation_phase"""
    w, _w = layer.w.array, layer._w.array
    assert w.ndim == 1
    n, = w.shape
    
    y0 = np.matmul(w.reshape(-1, 1), np.ones((1, n)))
    y1 = np.identity(n) - np.matmul(np.ones((n, 1)), w.reshape(1, -1))
    y2 = (1 - F.sigmoid(_w).array).reshape(-1, 1)
    grad_matrix = y0 * y1 * y2
    inv_grad_matrix = np.linalg.pinv(grad_matrix)
    
    return inv_grad_matrix @ layer._w.grad

def get_w_grad(layer):
    for p in layer._ws:
        if p.grad is not None:
            return p.grad

def print_grad(layer, enc_part):
    params = [
        layer.mu.grad.T, 
        sig_grad(layer).T,
        layer._w, 
        # w_grad(layer), 
        get_w_grad(layer),
    ]
    param_names = [
        "\u03BC", 
        "\u03C3", 
        "_w", 
        # "w*", 
        "w",
    ]
    _print(params, "grad", 
           headers=[f"\u2202 F{enc_part} / \u2202 {name}" for name in param_names])
    

# Base Plotter class

In [34]:

class Plotter(abc.ABC):
    __name__ = "Plotter"
    
    def __init__(self, max_comps=10, **kwargs):
        self.args = Args(**kwargs)
        print(self.args)
        self._X = None
        self.max_comps = max_comps
    
    @property
    def X(self):
        if self._X is None:
            self._X = new_data_grid(self.args)
        return self._X
        
    
    def __call__(self=None, 
                 mu=Args.init_mu, 
                 sig=Args.init_sig, 
                 N=Args.n_components):
        layer = new_layer(self.args, mu, sig, N)
        
        print_arrays(layer)
        self.plot(layer, self.X)
        
        #plt.tight_layout()
        plt.show()
        plt.close()
    
    def encode(self, layer, X):
        h, w, c = X.shape
        X = chainer.Variable(X)
        
        enc = layer(X.reshape(h*w, 1, c))
        enc = enc.reshape(h*w, 2, layer.n_components, layer.in_size)
        
        if layer.n_components > self.max_comps:
            print("More than {0} components! Plotting only the first {0}!".format(self.max_comps), 
                  file=sys.stderr)
            enc = enc[:, :, :self.max_comps]
        
        mu_enc, sig_enc = enc[:, 0], enc[:, 1]
        
        return X, mu_enc, sig_enc
        
    
    def interact(self, height=None,
                 mu=(-10, 10, 0.1), 
                 sig=(0.1, 100, 0.1),
                 N=(1, 32, 1)):
        _plot = interactive(self, mu=mu, sig=sig, N=N)
        
        # set fixed size if flickering is annoying
        if height is not None:
            output = _plot.children[-1]
            output.layout.height = f"{height}px"
        
        return _plot
        
    def plot(self, layer, X):
        fig, axs = plt.subplots(1,2, figsize=(16,8))
        fig.suptitle("Data and the GMM")
        
        layer.plot(ax=axs[1])
        axs[1].set_xlim(*self.args.x_range)
        axs[1].set_ylim(*self.args.y_range)
        ranges = self.args.x_range, self.args.y_range
        _imshow(axs[0], X, *ranges, title="Data")
        
        self.plot_core(layer, X)
        
    @abc.abstractmethod
    def plot_core(self, layer, X):
        pass

In [35]:
class EncodingPlotter(Plotter):
    
    def plot_core(self, layer, X):
        n0, n1, c = X.shape
        X, mu_enc, sig_enc = self.encode(layer, X)
            
        n_comp = mu_enc.shape[1]
        
        
        for i in np.arange(n_comp):
            
            fig, axs = plt.subplots(1, 2, figsize=(16,8))
            fig.suptitle(f"Component #{i}")
            fmt = "$\mathcal{{F}}_{{{param}_{{" + str(i) + "}}}}(x)$"
            for j, (_x, param) in enumerate(zip([mu_enc, sig_enc], ["\mu", "\sigma^2"])):
                title = fmt.format(param=param)
                _im = _x[:, i].reshape(n0, n1, 2)
                ranges = self.args.x_range, self.args.y_range
                _imshow(axs[j], _im, *ranges, title=title)

In [36]:
plotter = EncodingPlotter(
    eps=0,
)

plotter.interact()

Args(n_components=3, n_dims=2, init_mu=5, rnd_mu=True, init_sig=1, eps=0, x_range=(-10, 10), y_range=(-10, 10), n_samples=25, seed=42)


interactive(children=(FloatSlider(value=5.0, description='mu', max=10.0, min=-10.0), FloatSlider(value=1.0, de…

In [37]:
class GradPlotter(Plotter):
    
    def plot_core(self, layer, X):
       
        n0,n1,c = X.shape
        n0, n1, c = X.shape
        X, mu_enc, sig_enc = self.encode(layer, X)
        
        
        _title = f"All components"
        print(f"=== {_title} ===")
        title_fmt = "$\dfrac{{\partial}}{{\partial x}}\mathcal{{F}}_{{{param}}}(x)$" 
        
        enc_names = ["\mu", "\sigma^2"]#, "\Theta"]
        encs = [mu_enc, sig_enc]#, F.concat([mu_enc, sig_enc], axis=1)]
        
        fig, axs = self.plot_grad(layer, X, encs, enc_names, 
                                  no_grad_print=False,
                                  title_fmt=title_fmt)
        fig.suptitle(_title)
        
        
        for i in np.arange(mu_enc.shape[1]):
            
            title_fmt = "$\dfrac{{\partial}}{{\partial x}}\mathcal{{F}}_{{{param}_{{" + str(i) + "}}}}(x)$" 

            enc_names = ["\mu", "\sigma^2"]#, "\Theta"]
            encs = [mu_enc[:, i], sig_enc[:, i]]
            # encs.append(F.concat(encs, axis=-1))
            
            fig, axs = self.plot_grad(layer, X, encs, enc_names, 
                                      no_grad_print=True,
                                      title_fmt=title_fmt)

            _title = f"Component #{i}"
            fig.suptitle(_title)
        
    def plot_grad(self, layer, X, encs, enc_names, *, no_grad_print, title_fmt):
        
        fig, axs = plt.subplots(1, len(encs), figsize=(8*len(encs), 8))
        
        for ax, _enc, enc_name in zip(axs, encs, enc_names):
            X.grad = None
            layer.cleargrads()
            F.sum(_enc).backward(retain_grad=True)
            title = title_fmt.format(param=enc_name)
            if "mu" in enc_name:
                enc_part = "\u03BC"
                
            elif "sig" in enc_name:
                enc_part = "\u03C3"
                
            else:
                enc_part = "\u03B8"
            
            ranges = self.args.x_range, self.args.y_range
            _imshow(ax, X.grad, *ranges, title=title)    
            
            
            if no_grad_print:
                continue
            
            print_grad(layer, 
                enc_part="\u03BC" if "mu" in enc_name else "\u03C3" if "sig" in enc_name else "\u03B8" )
            
        return fig, axs
        

In [ ]:

plotter = GradPlotter(
    eps=0,
    rnd_mu=True,
)

plotter.interact()

Args(n_components=3, n_dims=2, init_mu=5, rnd_mu=True, init_sig=1, eps=0, x_range=(-10, 10), y_range=(-10, 10), n_samples=25, seed=42)


interactive(children=(FloatSlider(value=5.0, description='mu', max=10.0, min=-10.0), FloatSlider(value=1.0, de…

In [ ]:
layer = new_layer(plotter.args)

layer._sig, layer.sig